> copied from [tl2cents](https://github.com/tl2cents/CTF-Writeups/blob/master/2025/ACTF/AAALLL/exp2.py)

In [1]:
from sage.all import block_matrix, GF, ZZ, vector, matrix, save, load
from subprocess import check_output
from re import findall
import os
with open("output.txt", "r") as f:
    exec(f.read())
n = 512
t = n//2

def flatter(M):
    # compile https://github.com/keeganryan/flatter and put it in $PATH
    z = "[[" + "]\n[".join(" ".join(map(str, row)) for row in M) + "]]"
    env = os.environ.copy()
    env["OMP_NUM_THREADS"] = "8"  # macbook上控制线程数量, 避免调度到小核上
    ret = check_output(["flatter"], input=z.encode(), env=env)
    return matrix(M.nrows(), M.ncols(), map(int, findall(rb"-?\d+", ret)))

def modulo_reduction(M, p, verbose=False):
    """ Perform LLL reduction on the matrix M with modulo p. 
    An optimized version, see this blog for more details: https://blog.tanglee.top/2024/07/15/HITCON-CTF-2024-Qual-Crypto-Writeup.html#implementation

    Args:
        M (matrix): the matrix to reduce
        p (int): the modulo
        verbose (bool, optional): whether to print the debug information. Defaults to False.

    Returns:
        matrix: The reduced matrix
    """
    n, m = M.nrows(), M.ncols()
    if n < m:
        Me = M.change_ring(GF(p)).echelon_form()
        delta = Me.ncols() - n
        zero_mat = matrix.zero(delta, n)
        pI = matrix.identity(delta) * p
        L = block_matrix(ZZ, [[Me], [zero_mat.augment(pI)]])
        if L.rank() != L.nrows():
            L = L.echelon_form(algorithm="pari0", include_zero_rows=False, include_zero_columns=False)
        L = L.change_ring(ZZ)
    else:
        pI = matrix.identity(m) * p
        L = block_matrix(ZZ, [[M], [pI]])
        
    if verbose:
        print(f"Starts to do LLL reduction with dimensions {L.dimensions()}")
    try:
        L = flatter(L)
        # L = L.LLL()
    except Exception as e:
        print(f"Unable to use flatter: {e}")
        exit(1)
    if verbose: print(f"LLL done😊")
    return L

assert len(points) == n // 2
rows = []
for i in range(n):
    rows.append([r**i for r in points])

# we turn the original problem into an instance of SIS problem
M = matrix(GF(p), rows).T
# we use LLL to search the short vector in the kernel space of M
ker = list(M.right_kernel().basis())
sol = M.solve_right(vector(GF(p), results))
print(f"ker: {len(ker) = }")
M0 = matrix(ZZ, ker + [sol])
L = modulo_reduction(M0, p, verbose=True)
save(L, "./L2.sobj")

ker: len(ker) = 256
Starts to do LLL reduction with dimensions (512, 512)
LLL done😊


In [2]:
L = load("./L2.sobj")
for row in L:
    if all(num in [-1, 1] for num in row):
        print("".join(str(-c) for c in list(row)[::-1]))
        continue

-1-1-111-111-11-11-111-1111-11-1-1-1-111-1-1-1-1-1-1-111-1-1-11-1-11-1-11-1-11-11-11-1-1-11-1-11-1-111-11-11-1-1-1-11111-1-1-111-111111-11-1-11111-111111111-1-1111-1-1-1-1-1111111-11-1-111-11111111-11-1-1-1-11-1-1-11111-1-1-1-11-1-11-1-1-1-11-1-1-111-1-1-11-1-1-11-1-1-11-1-1-1-1-1-11-1111-1-1-1-1-1111-11-1-11-1-111-1-1-1-11-1-11-1-1-1-1-111-1-1-1-1-1-111-111-1-1-1111-1-1-11-1-1111-1-1-1-1-1111-111-11-11111-11-111-1-1-111111-1-1-11-11-11-1-1111-1-1-11-111-111111-11-11111-11-1-1111-1-11-11-1-1111-1-111-1-1111-11-111-1-111111-1-11-111111-111-1-1-1-1-11-1-1-111-11-11-111-1-1-1-111-1-1-111-1-11-1-1-1111-1-111-11-11-11-1-1-1-11-11-11-11-11-111111-11-1-1-1-1-1-11111-1-11-11-11111-11-11-111-1-1111-111-11-1-111-1-11-1-1-1-11-11-11-1-11111-1-1-1-111-1-1-1-111-1-1111-11-1-11-1


In [3]:
for row in L:
    if all(num in [-1, 1] for num in row):
        print("".join(str(c) for c in list(row)[::-1]))
        continue

111-1-11-1-11-11-11-1-11-1-1-11-11111-1-11111111-1-1111-111-111-111-11-11-1111-111-111-1-11-11-11111-1-1-1-1111-1-11-1-1-1-1-11-111-1-1-1-11-1-1-1-1-1-1-1-111-1-1-111111-1-1-1-1-1-11-111-1-11-1-1-1-1-1-1-11-11111-1111-1-1-1-11111-111-11111-1111-1-1111-1111-1111-1111111-11-1-1-111111-1-1-11-111-111-1-11111-111-111111-1-1111111-1-11-1-1111-1-1-1111-111-1-1-111111-1-1-11-1-11-11-1-1-1-11-11-1-1111-1-1-1-1-1111-11-11-111-1-1-1111-11-1-11-1-1-1-1-11-11-1-1-1-11-111-1-1-111-11-111-1-1-111-1-111-1-1-11-11-1-111-1-1-1-1-111-11-1-1-1-1-11-1-111111-1111-1-11-11-11-1-11111-1-1111-1-111-1111-1-1-111-1-11-11-11-11111-11-11-11-11-11-1-1-1-1-11-1111111-1-1-1-111-11-11-1-1-1-11-11-11-1-111-1-1-11-1-11-111-1-111-11111-11-11-111-1-1-1-11111-1-11111-1-111-1-1-11-111-11
